Llama Index Dispatcher context fields are passed through to workflow runs. Additionally, workflow runs track their individual `run_id`s as a context field. Tracking these fields in logs can be useful to differentiate runs when running with concurrency, and associate them back to a trace.

This notebook demonstrates how to integrate with standard library logging, as well as with structlog, for including these fields in logs.

In [ ]:
%pip install structlog llama-index-workflows

Set up imports

In [ ]:
import logging
from typing import Any, MutableMapping
import structlog

from llama_index_instrumentation.dispatcher import (
    active_instrument_tags,
    instrument_tags,
)

from workflows import Context, Workflow, step
from workflows.events import StartEvent, StopEvent

set up structlog to read from the dispatcher context:

In [16]:
def merge_custom_context(
    _logger: structlog.BoundLogger,
    _method_name: str,
    event_dict: MutableMapping[str, Any],
) -> MutableMapping[str, Any]:
    """
    Merge values from your ContextVar dict into structlog's event_dict.
    Later processors (e.g., JSONRenderer) will see these keys as if bound.
    """
    ctx = active_instrument_tags.get()
    if ctx:
        # don't clobber explicitly-set event keys unless you want to:
        for k, v in ctx.items():
            event_dict.setdefault(k, v)
            # or: event_dict[k] = v  # if you want your ctx to win
    return event_dict


structlog.configure(
    processors=[
        merge_custom_context,  # <------------- Add this to add llama index dispatcher tags to structlog
        structlog.processors.add_log_level,
        structlog.processors.TimeStamper(fmt="%Y-%m-%d %H:%M:%S", utc=False),
        structlog.dev.ConsoleRenderer(),
    ],
)

Set up structlog to read from the dispatcher context:

In [2]:
def merge_custom_context(
    _logger: structlog.BoundLogger,
    _method_name: str,
    event_dict: MutableMapping[str, Any],
) -> MutableMapping[str, Any]:
    """
    Merge values from your ContextVar dict into structlog's event_dict.
    Later processors (e.g., JSONRenderer) will see these keys as if bound.
    """
    ctx = active_instrument_tags.get()
    if ctx:
        # don't clobber explicitly-set event keys unless you want to:
        for k, v in ctx.items():
            event_dict.setdefault(k, v)
            # or: event_dict[k] = v  # if you want your ctx to win
    return event_dict


structlog.configure(
    processors=[
        merge_custom_context,  # <------------- Add this to add llama index dispatcher tags to structlog
        structlog.processors.add_log_level,
        structlog.processors.TimeStamper(fmt="%Y-%m-%d %H:%M:%S", utc=False),
        structlog.dev.ConsoleRenderer(),
    ],
)

Set up stdlib logging to include the run_id from the dispatcher context. Note that stdlib logging is much harder to configure correctly, and has difficulty with extra fields being optional or overwritten.

In [6]:
old_factory = logging.getLogRecordFactory()


def record_factory(*args, **kwargs):
    record = old_factory(*args, **kwargs)  # get the unmodified record
    record.run_id = active_instrument_tags.get().get("run_id", "")
    return record


logging.setLogRecordFactory(record_factory)

logging.basicConfig(level=logging.INFO, format="%(message)s run_id=%(run_id)s")

In [7]:
structlog_logger = structlog.get_logger()
regular_logger = logging.getLogger()
structlog_logger.info("Hello from structlog")
regular_logger.info("Hello from stdlib")

2025-11-05 22:48:59 [info     ] Hello from structlog          


Hello from stdlib run_id=


Set up an example workflow that demonstrates log context:

In [10]:
class LoggingWorkflow(Workflow):
    """A workflow that demonstrates log context."""

    @step
    async def log_step(self, ctx: Context, ev: StartEvent) -> StopEvent:
        # Any fields bound here will also appear alongside dispatcher tags
        structlog_logger.info("structlog processing step")
        # Without a more complex wrappers, the fields must be manually passed into standard logging
        regular_logger.info("regular processing step")

        return StopEvent(result="ok")

And run it! Try multiple times and see the run_id change.

In [15]:
# Tags set outside the workflow run will be captured in all logs emitted
# during the run (together with run_id injected by the broker).
wf = LoggingWorkflow()

with instrument_tags({"request_id": "req-123", "user": "alice"}):
    result = await wf.run()
structlog_logger.info(f"final result '{result}'")

2025-11-05 22:51:47 [info     ] structlog processing step      request_id=req-123 run_id=lBUAX17ywM user=alice


regular processing step run_id=lBUAX17ywM


2025-11-05 22:51:47 [info     ] final result 'ok'             
